INDEXING

In [163]:
import os
from PyPDF2 import PdfReader

In [164]:
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)


In [165]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
markdown_path = r"C:\Users\koolv\Mr.Poopybutthole\Eren's RAG\Data\ncert.md"
loader = UnstructuredMarkdownLoader(markdown_path)
document = loader.load()
type(document)

list

In [166]:
document = str(document)

Number of Tokens

In [167]:
import tiktoken

In [168]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens
num_tokens_from_string(document, "cl100k_base")

161688

Splitter

In [169]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=10000, chunk_overlap= 5000)
split = text_splitter.split_text(document)
print(split)

["[Document(page_content='Chemistry\\n\\nPart I\\n\\nTextbook for Class XI\\n\\n2 CHEMISTRY\\n\\nANSWERS 3\\n\\nFOREWORD\\n\\nThe National Curriculum Framework (NCF), 2005 recommends that children’s life at\\nschool must be linked to their life outside the school. This principle marks a departure\\nfrom the legacy of bookish learning which continues to shape our system and causes a\\ngap between the school, home and community. The syllabi and textbooks developed on\\nthe basis of NCF signify an attempt to implement this basic idea. They also attempt to\\ndiscourage rote learning and the maintenance of sharp boundaries between different\\nsubject areas. We hope these measures will take us significantly further in the direction of\\na child-centred system of education outlined in the National Policy on Education (1986).\\nThe success of this effort depends on the steps that school principals and teachers\\nwill take to encourage children to reflect on their own learning and to pursue\\ni

TEXT EMBEDDING

In [170]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
document_result = embeddings.embed_query(split)
document_result[:5]

[0.020169366, -0.0038486598, -0.029890932, -0.0074242475, 0.07558129]

Vectorstores

In [171]:
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(
                     documents=text_splitter.create_documents(split),
                     embedding=embeddings,    # Embedding model
                     persist_directory="./chroma_db" # Directory to save data
                     )

In [172]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
docs = retriever.get_relevant_documents('what is atom')

In [173]:
len(docs)

1

In [174]:
print(docs)

[Document(page_content='2.1 (i)1.099 × 1027\n electrons (ii)5.48 × 10–7 kg, 9.65 × 104C\n2.2 (i)6.022 × 1024electrons\n(ii)(a)  2.4088 × 1021 neutrons (b)  4.0347 × 10–6 kg\n(iii)(a)  1.2044 × 1022 protons (b)  2.015  × 10–5 kg\n2.3 7,6: 8,8: 12,12: 30,26: 50, 38\n2.4 (i)Cl (ii)U (iii)Be\n2.5 5.17 × 1014 s–1, 1.72 × 106m–1\n2.6 (i)1.988 × 10–18 J (ii)3.98 × 10–15 JAnswer to Some Selected Problems\n2018-19\n254 CHEMISTRY\n2.7 6.0 × 10–2 m,  5.0 × 109 s–1 and  16.66 m–1\n2.8 2.012 × 1016 photons\n2.9 (i) 4.97 × 10–19 J (3.10 eV); (ii)0.97 eV (iii)5.84 × 105 m s–1\n2.10 494 kJ mol–1\n2.11 7.18 × 1019s–1\n2.12 4.41 × 1014s–1, 2.91 × 10–19J\n2.13 486 nm\n2.14 8.72 × 10–20J\n2.15 15 emission lines\n2.16 (i)8.72 × 10–20J (ii)1.3225 nm\n2.17 1.523 × 106 m–1\n2.18 2.08 × 10–11 ergs, 950 \uf0c5\n2.19 3647\uf0c5\n2.20 3.55 × 10–11m\n2.21 8967\uf0c5\n2.22 Na+, Mg2+, Ca2+; Ar, S2– and  K+\n2.23 (i)  (a)  1s2   (b)  1s2 2s2 2p6;   (c)  1s22s22p6   (d)  1s22s22p6\n2.24 n = 5\n2.25 n = 3; l = 2; ml = 

Generation

In [175]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate

In [176]:
template = """Answer the question in the context of this
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question in the context of this\n{context}\nQuestion: {question}\n'))])

In [177]:
llm = ChatGoogleGenerativeAI(model = 'gemini-pro')

In [178]:
chain = prompt | llm
chain.invoke({"context":docs,"question":"What is atom?"})

AIMessage(content='This document does not mention the definition of an atom.', response_metadata={'prompt_feedback': {'safety_ratings': [{'category': 9, 'probability': 1, 'blocked': False}, {'category': 8, 'probability': 1, 'blocked': False}, {'category': 7, 'probability': 1, 'blocked': False}, {'category': 10, 'probability': 1, 'blocked': False}], 'block_reason': 0}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]})